In [ ]:
from google.colab import files
uploaded = files.upload()

Saving labelled_train_set (1).csv to labelled_train_set (1).csv


In [ ]:
import pandas as pd

train_data = pd.read_csv('labelled_train_set (1).csv')
train_data

,ID,Article,Type
0,TRAIN_1,There are a variety of emerging applications f...,Human-written
1,TRAIN_2,As each new means of communication and social ...,Human-written
2,TRAIN_3,"These suggestions include:, Learn about the pu...",Human-written
3,TRAIN_4,In recent years there has been growing concern...,Human-written
4,TRAIN_5,"By the late 1970s, researchers at many major u...",Human-written
...,...,...,...
495,TRAIN_496,Handwriting recognition technology has numerou...,AI-generated
496,TRAIN_497,Digital Rights Management (DRM) (sometimes als...,Human-written
497,TRAIN_498,A cookie is a small piece of data that is stor...,AI-generated
498,TRAIN_499,The hexadecimal system is a number system with...,AI-generated


In [ ]:
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
from imblearn.over_sampling import SMOTE
import joblib
import nltk

nltk.download('stopwords')
nltk.download('punkt')

def clean_text(text):
    text = text.lower()
    text = re.sub(r'[^\w\s]', '', text)
    words = word_tokenize(text)
    stop_words = set(stopwords.words('english'))
    cleaned_words = [word for word in words if word not in stop_words]
    return ' '.join(cleaned_words)


train_data = pd.read_csv('labelled_train_set (1).csv')


train_data['clean_article'] = train_data['Article'].apply(clean_text)


vectorizer = TfidfVectorizer(max_features=5000)
X_train = vectorizer.fit_transform(train_data['clean_article'])


label_mapping = {'Human-written': 0, 'AI-generated': 1}
y_train = train_data['Type'].map(label_mapping)


smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X_train, y_train)


print(f"Class distribution before SMOTE:\n{y_train.value_counts()}")
print(f"Class distribution after SMOTE:\n{pd.Series(y_resampled).value_counts()}")


model = LogisticRegression(max_iter=1000)
model.fit(X_resampled, y_resampled)


joblib.dump(model, 'model.pkl')
joblib.dump(vectorizer, 'vectorizer.pkl')


X_train_final = vectorizer.transform(train_data['clean_article'])


y_pred_final = model.predict(X_train_final)


train_data['Predicted_Type'] = y_pred_final
train_data.to_csv('predictions.csv', index=False)


if 'Type' in train_data.columns:
    y_test_final = train_data['Type'].map(label_mapping)
    accuracy_final = accuracy_score(y_test_final, y_pred_final)
    report_final = classification_report(y_test_final, y_pred_final)

    print(f"Test Accuracy: {accuracy_final}")
    print("Test Classification Report:")
    print(report_final)
else:
    print("No true labels ('Type' column) found in the test dataset. Skipping evaluation.")


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Class distribution before SMOTE:
Type
0    386
1    114
Name: count, dtype: int64
Class distribution after SMOTE:
Type
0    386
1    386
Name: count, dtype: int64
Test Accuracy: 0.996
Test Classification Report:
              precision    recall  f1-score   support

           0       1.00      0.99      1.00       386
           1       0.98      1.00      0.99       114

    accuracy                           1.00       500
   macro avg       0.99      1.00      0.99       500
weighted avg       1.00      1.00      1.00       500



In [ ]:
from google.colab import files
uploaded = files.upload()

Saving unlabelled_test2.csv to unlabelled_test2.csv


In [ ]:
import pandas as pd

test_data = pd.read_csv('unlabelled_test2.csv')
test_data

,ID,Article
0,TEST_1,Grid computing is a type of distributed comput...
1,TEST_2,"RAID stands for ""Redundant Array of Independen..."
2,TEST_3,A computer password is a sequence of character...
3,TEST_4,For people who find commercial online services...
4,TEST_5,A Green PC is a computer that has been designe...
...,...,...
512,TEST_513,A Computer Security Incident Response Team (CS...
513,TEST_514,"Recordable CDs, also known as CD-Rs, are a typ..."
514,TEST_515,A webmaster is a person responsible for managi...
515,TEST_516,The standard for serial transmission is a set ...


In [ ]:
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
import joblib

def clean_text(text):
    text = text.lower()
    text = re.sub(r'[^\w\s]', '', text)
    words = word_tokenize(text)
    stop_words = set(stopwords.words('english'))
    cleaned_words = [word for word in words if word not in stop_words]
    return ' '.join(cleaned_words)

test_data = pd.read_csv('unlabelled_test2.csv')


test_data['clean_article'] = test_data['Article'].apply(clean_text)


model = joblib.load('model.pkl')
vectorizer = joblib.load('vectorizer.pkl')

X_test_final = vectorizer.transform(test_data['clean_article'])


y_pred_final = model.predict(X_test_final)


test_data['Predicted_Type'] = y_pred_final


test_data = test_data.drop(columns=['Article', 'clean_article'])


test_data.to_csv('test2_set_predictions.csv', index=False)
